# Reading HDF5 files to textfiles for later use in Julia

This is a stupid way of doing this. It's a little like making sauce in one pot and then transfering it to a another pot just before serving. Sure, you can do that, but the only thing you gain is more dishes and an increased likelyhood of burning yourself.

At the time I was unable to read the HDF5 files in Julia.

In [1]:
import numpy
from helita.sim import rh15d
from tqdm import tqdm

In [2]:
# Directories with HDF5 data 
FALC_in = "/mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhInput/FALC/FALC_82_5x5.hdf5"
FALC_out = "/mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/FALC/Experiment2"

Bifrost_out = "/mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/Bifrost/Experiment3"
Bifrost_in = "/mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhInput/Bifrost/bifrost_cb24bih_s385.hdf5"

# Directory to place textfiles
path = "/mn/stornext/d18/RoCS/idarhan/MScProject_basement/Renditions/R1/data/"

In [3]:
# Use helita to read output 
data_FALC = rh15d.Rh15dout(FALC_out)
data_Bifrost = rh15d.Rh15dout(Bifrost_out)

/astro/local/anaconda/envs/py38/lib/python3.8/site-packages/helita/sim/rh15d.py:40: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  setattr(self, g, xr.open_dataset(infile, group=g, autoclose=True))
/astro/local/anaconda/envs/py38/lib/python3.8/site-packages/helita/sim/rh15d.py:53: FutureWarning: The autoclose argument is no longer used by xarray.open_dataset() and is now ignored; it will be removed in a future version of xarray. If necessary, you can control the maximum number of simultaneous open files with xarray.set_options(file_cache_maxsize=...).
  self.ray = xr.open_dataset(infile, autoclose=True)


--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/FALC/Experiment2/output_aux.hdf5 file.
--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/FALC/Experiment2/output_indata.hdf5 file.
--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/FALC/Experiment2/output_ray.hdf5 file.
--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/Bifrost/Experiment3/output_aux.hdf5 file.
--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/Bifrost/Experiment3/output_indata.hdf5 file.
--- Read /mn/stornext/d18/RoCS/idarhan/MScProject_basement/OpThickB/rhOutput/Bifrost/Experiment3/output_ray.hdf5 file.


In [4]:
# Remove NAN entries, to save time and memory in I/O
FALC_atmos = data_FALC.atmos.dropna('height')
FALC_ray = data_FALC.ray.dropna('height') 

In [5]:
index500 = numpy.argmin(numpy.asarray(abs(FALC_ray['wavelength_selected'] - 500)))
index500_full =  numpy.argmin(numpy.asarray(abs(FALC_ray['wavelength'] - 500)))

In [8]:
# Load relevant parameters 
x_FALC = FALC_atmos['x']
y_FALC = FALC_atmos['y']
height_FALC = FALC_atmos['height_scale'][0,0]
temperature_FALC = FALC_atmos['temperature']
electron_density_FALC = FALC_atmos['electron_density']
velocity_z_FALC = FALC_atmos['velocity_z']

chi_continuum_FALC = FALC_ray['chi_continuum'][:,:,:,index500]
epsilon_continuum_FALC = FALC_ray['epsilon_continuum'][:,:,:,index500]
intensity_FALC = FALC_ray['intensity'][:,:,index500_full]

In [12]:
# Remove NAN entries, to save time and memory in I/O
# but not on ray data, because time-consuming and not going to use most data
Bifrost_atmos = data_Bifrost.atmos.dropna('height')
Bifrost_ray = data_Bifrost.ray 

In [8]:
# Load relevant parameters 
x_Bifrost = Bifrost_atmos['x']
y_Bifrost = Bifrost_atmos['y']
height_Bifrost = Bifrost_atmos['height_scale'][0,0]
temperature_Bifrost = Bifrost_atmos['temperature']
electron_density_Bifrost = Bifrost_atmos['electron_density']
velocity_z_Bifrost = Bifrost_atmos['velocity_z']

# Remove NAN from sliced arrays 
chi_continuum_Bifrost = Bifrost_ray['chi_continuum'][:,:,:,index500].dropna('height')
epsilon_continuum_Bifrost = Bifrost_ray['epsilon_continuum'][:,:,:,index500].dropna('height')
intensity_Bifrost = Bifrost_ray['intensity'][:,:,index500_full]

In [8]:
def writeToFile(parameter, unit, filename):
    """
    Function for writing a 1D, 2D or 3D array to a textfile. 
    
    First line gives unit
    """
    
    f = open(filename, 'w')
    
    shape = parameter.shape
    dim = len(shape)
    
    f.write(unit)
    f.write("\n")
    f.write(str(shape))
    
    if dim == 1:
        f.write("\n\n")
        for i in range(shape[0]):
            
            f.write("%e " %parameter[i])

    elif dim == 2:
        f.write("\n")
        for j in range(shape[1]):
            f.write("\n")
            for i in range(shape[0]):
                f.write("%e " %parameter[i,j]) 
    
    elif dim == 3:
        
        for k in tqdm(range(shape[2])):
            f.write("\n")
            for j in range(shape[1]):
                f.write("\n")
                for i in range(shape[0]):
                    f.write("%e " %parameter[i,j,k]) 
        
    else:
        print("Can only read 1D or 3D parameters. Check the shape of your input.")    
    
    f.close()

In [14]:
writeToFile(x_FALC, 'm', path + 'x_FALC.txt')
writeToFile(y_FALC, 'm', path + 'y_FALC.txt')
writeToFile(height_FALC, 'm', path + 'height_FALC.txt')
writeToFile(temperature_FALC, 'K', path + 'temperature_FALC.txt')
writeToFile(electron_density_FALC, 'm-3', path + 'electron_density_FALC.txt')
writeToFile(velocity_z_FALC, 'm s-1', path + 'velocity_z_FALC.txt')

writeToFile(chi_continuum_FALC, 'm^-1', path + 'chi_continuum_FALC.txt')
writeToFile(epsilon_continuum_FALC, '1', path + 'epsilon_continuum_FALC.txt')
writeToFile(intensity_FALC, 'W m^-2 Hz^-1 sr^-1', path + 'intensity_FALC.txt')

100%|██████████| 68/68 [00:00<00:00, 187.94it/s]


In [10]:
writeToFile(x_Bifrost, 'm', path + 'x_Bifrost.txt')
writeToFile(y_Bifrost, 'm', path + 'y_Bifrost.txt')
writeToFile(height_Bifrost, 'm', path + 'height_Bifrost.txt')
writeToFile(temperature_Bifrost, 'K', path + 'temperature_Bifrost.txt')
writeToFile(electron_density_Bifrost, 'm-3', path + 'electron_density_Bifrost.txt')
writeToFile(velocity_z_Bifrost, 'm s-1', path + 'velocity_z_Bifrost.txt')

writeToFile(chi_continuum_Bifrost, 'm^-1', path + 'chi_continuum_Bifrost.txt')
writeToFile(epsilon_continuum_Bifrost, '1', path + 'epsilon_continuum_Bifrost.txt')
writeToFile(intensity_Bifrost, 'W m^-2 Hz^-1 sr^-1', path + 'intensity_Bifrost.txt')

100%|██████████| 117/117 [1:46:40<00:00, 54.70s/it]
